In [1]:
import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from runpy import run_path
from skimage import img_as_ubyte
from natsort import natsorted
from glob import glob
import cv2
from tqdm import tqdm
import argparse
import numpy as np
import os
from glob import glob
import shutil
from natsort import natsorted
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import skimage.metrics
from torchvision import models
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from PIL import Image

## Restormer

In [2]:
!git clone https://github.com/swz30/Restormer.git
%cd Restormer
!wget https://github.com/swz30/Restormer/releases/download/v1.0/real_denoising.pth -P Denoising/pretrained_models

Cloning into 'Restormer'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (309/309), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 309 (delta 122), reused 261 (delta 113), pack-reused 0 (from 0)
Receiving objects: 100% (309/309), 1.56 MiB | 14.13 MiB/s, done.
Resolving deltas: 100% (122/122), done.
/content/Restormer
--2024-11-02 16:31:51--  https://github.com/swz30/Restormer/releases/download/v1.0/real_denoising.pth
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/418793252/577ea2a7-8cf3-44b2-900d-5368f402de29?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20241102%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20241102T163151Z&X-Amz-Expires=300&X-Amz-Signature=7222f3067ba7fb414fac4582864f3cf4b168a2befaf9e13

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def get_weights_and_parameters(task, parameters):
    if task == 'Motion_Deblurring':
        weights = os.path.join('Motion_Deblurring', 'pretrained_models', 'motion_deblurring.pth')
    elif task == 'Single_Image_Defocus_Deblurring':
        weights = os.path.join('Defocus_Deblurring', 'pretrained_models', 'single_image_defocus_deblurring.pth')
    elif task == 'Deraining':
        weights = os.path.join('Deraining', 'pretrained_models', 'deraining.pth')
    elif task == 'Real_Denoising':
        weights = os.path.join('Denoising', 'pretrained_models', 'real_denoising.pth')
        parameters['LayerNorm_type'] =  'BiasFree'
    return weights, parameters

root_dir = '/content/Restormer'
# Get model weights and parameters
task = 'Real_Denoising'
parameters = {'inp_channels':3, 'out_channels':3, 'dim':48, 'num_blocks':[4,6,6,8], 'num_refinement_blocks':4, 'heads':[1,2,4,8], 'ffn_expansion_factor':2.66, 'bias':False, 'LayerNorm_type':'WithBias', 'dual_pixel_task':False}
weights, parameters = get_weights_and_parameters(task, parameters)

load_arch = run_path(os.path.join('basicsr', 'models', 'archs', 'restormer_arch.py'))
model = load_arch['Restormer'](**parameters)
model.cuda()

checkpoint = torch.load(os.path.join(root_dir, weights))
model.load_state_dict(checkpoint['params'])
model.eval()

<ipython-input-4-6483d258c817>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(root_dir, weights))


Restormer(
  (patch_embed): OverlapPatchEmbed(
    (proj): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (encoder_level1): Sequential(
    (0): TransformerBlock(
      (norm1): LayerNorm(
        (body): BiasFree_LayerNorm()
      )
      (attn): Attention(
        (qkv): Conv2d(48, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (qkv_dwconv): Conv2d(144, 144, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=144, bias=False)
        (project_out): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (norm2): LayerNorm(
        (body): BiasFree_LayerNorm()
      )
      (ffn): FeedForward(
        (project_in): Conv2d(48, 254, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (dwconv): Conv2d(254, 254, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=254, bias=False)
        (project_out): Conv2d(127, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (1): TransformerBlock(
 

In [5]:
%cd ..

/content


In [6]:
!unzip /content/drive/MyDrive/Denoising_Dataset_train_val.zip

Archive:  /content/drive/MyDrive/Denoising_Dataset_train_val.zip
   creating: Denoising_Dataset_train_val/
   creating: Denoising_Dataset_train_val/bottle/
   creating: Denoising_Dataset_train_val/bottle/Train/
   creating: Denoising_Dataset_train_val/bottle/Train/Defect_mask/
   creating: Denoising_Dataset_train_val/bottle/Train/Defect_mask/broken_large/
  inflating: Denoising_Dataset_train_val/bottle/Train/Defect_mask/broken_large/001_mask.png  
  inflating: Denoising_Dataset_train_val/bottle/Train/Defect_mask/broken_large/002_mask.png  
  inflating: Denoising_Dataset_train_val/bottle/Train/Defect_mask/broken_large/003_mask.png  
  inflating: Denoising_Dataset_train_val/bottle/Train/Defect_mask/broken_large/004_mask.png  
  inflating: Denoising_Dataset_train_val/bottle/Train/Defect_mask/broken_large/006_mask.png  
  inflating: Denoising_Dataset_train_val/bottle/Train/Defect_mask/broken_large/007_mask.png  
  inflating: Denoising_Dataset_train_val/bottle/Train/Defect_mask/broken_large

In [7]:
def process_images(input_dir, output_dir='/content/denoised_test_images', img_multiple_of=8):
    for class_name in os.listdir(input_dir):
        class_dir = os.path.join(input_dir, class_name, "Val")
        degraded_subdirs = glob(os.path.join(class_dir, "Degraded_image", "*"))  # Capture all subclasses

        print(f"\n ==> Processing class: {class_name} with {len(degraded_subdirs)} subclasses\n ")

        for degraded_subdir in degraded_subdirs:
            degraded_paths = glob(os.path.join(degraded_subdir, "*.png"))

            # Create output directory for the subclass
            relative_subdir = os.path.relpath(degraded_subdir, os.path.join(class_dir, "Degraded_image"))
            output_subdir = os.path.join(output_dir, class_name, "Train", "Degraded_image", relative_subdir)
            os.makedirs(output_subdir, exist_ok=True)

            with torch.no_grad():
                for degraded_path in tqdm(degraded_paths):
                    # Load degraded image
                    img = cv2.cvtColor(cv2.imread(degraded_path), cv2.COLOR_BGR2RGB)
                    input_ = torch.from_numpy(img).float().div(255.).permute(2, 0, 1).unsqueeze(0).cuda()

                    # Pad the input if not a multiple of 8
                    h, w = input_.shape[2], input_.shape[3]
                    H, W = ((h + img_multiple_of) // img_multiple_of) * img_multiple_of, ((w + img_multiple_of) // img_multiple_of) * img_multiple_of
                    padh = H - h if h % img_multiple_of != 0 else 0
                    padw = W - w if w % img_multiple_of != 0 else 0
                    input_ = F.pad(input_, (0, padw, 0, padh), 'reflect')

                    # Denoise using the Restormer model
                    restored = model(input_)
                    restored = torch.clamp(restored, 0, 1)

                    # Unpad the output
                    restored = restored[:, :, :h, :w]
                    restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
                    restored = img_as_ubyte(restored[0])

                    # Save the denoised image in the output directory
                    output_image_path = os.path.join(output_subdir, os.path.basename(degraded_path))
                    cv2.imwrite(output_image_path, cv2.cvtColor(restored, cv2.COLOR_RGB2BGR))

In [8]:
process_images(input_dir='/content/Denoising_Dataset_train_val', output_dir='/content/denoised_test_images', img_multiple_of=8)


 ==> Processing class: capsule with 5 subclasses
 


100%|██████████| 1/1 [00:03<00:00,  3.38s/it]



 ==> Processing class: transistor with 4 subclasses
 


100%|██████████| 1/1 [00:03<00:00,  3.52s/it]



 ==> Processing class: toothbrush with 1 subclasses
 


...

100%|██████████| 1/1 [00:03<00:00,  3.78s/it]



 ==> Processing class: hazelnut with 4 subclasses
 


100%|██████████| 1/1 [00:03<00:00,  3.85s/it]



 ==> Processing class: bottle with 3 subclasses
 


100%|██████████| 2/2 [00:06<00:00,  3.05s/it]


## Custom Model for Reconstruction

In [9]:
!wget https://github.com/uncertainty3/EE5179_Project_Jul-Nov-24/blob/main/final_mode_vggresnet.pth -P /content

--2024-11-02 16:38:45--  https://github.com/uncertainty3/EE5179_Project_Jul-Nov-24/blob/main/final_mode_vggresnet.pth
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/final_mode_vggresnet.pth’

final_mode_vggresne     [ <=>                ] 283.87K  --.-KB/s    in 0.03s   

2024-11-02 16:38:46 (9.60 MB/s) - ‘/content/final_mode_vggresnet.pth’ saved [290680]



In [10]:
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.conv1(x))
        out = self.conv2(out)
        return out + residual

class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1, padding=0)
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1, padding=0)

    def forward(self, x):
        out = self.avg_pool(x)
        out = F.relu(self.fc1(out))
        out = torch.sigmoid(self.fc2(out))
        return x * out

class SRModel_Complex(nn.Module):
    def __init__(self, num_classes):
        super(SRModel_Complex, self).__init__()
        self.num_classes = num_classes
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.class_embedding = nn.Embedding(num_classes, 64)
        self.residual_block1 = ResidualBlock(64)
        self.residual_block2 = ResidualBlock(64)
        self.attention = ChannelAttention(64)
        self.conv2 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 3, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x, class_idx):
        x = self.relu(self.conv1(x))

        # Class embedding for multi-task learning
        class_emb = self.class_embedding(class_idx).unsqueeze(-1).unsqueeze(-1)
        class_emb = class_emb.expand(-1, -1, x.shape[2], x.shape[3])

        x = torch.cat([x, class_emb], dim=1)
        x = self.relu(self.conv2(x))

        # Residual and Attention Layers
        x = self.residual_block1(x)
        x = self.residual_block2(x)
        x = self.attention(x)

        # Final output
        x = self.conv3(x)
        return x


class PerceptualLoss(nn.Module):
    def __init__(self, layer_weights=None):
        """
        Initialize the perceptual loss function by loading a pre-trained VGG19 model.

        Args:
            layer_weights (dict): Weights for each layer to balance perceptual loss contributions.
        """
        super(PerceptualLoss, self).__init__()
        vgg = models.vgg19(pretrained=True).features
        self.layers = nn.ModuleList([
            vgg[:4],   # Conv1_2
            vgg[:9],   # Conv2_2
            vgg[:18],  # Conv3_4
            vgg[:27]   # Conv4_4
        ])
        self.layer_weights = layer_weights if layer_weights else [1.0, 0.75, 0.5, 0.25]

        # Freeze VGG parameters
        for param in self.layers.parameters():
            param.requires_grad = False

    def forward(self, input, target):
        loss = 0.0
        for i, layer in enumerate(self.layers):
            input_features = layer(input)
            target_features = layer(target)
            loss += self.layer_weights[i] * nn.functional.mse_loss(input_features, target_features)
        return loss

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SRModel_Complex(num_classes=15)  # Instantiate your model
model.load_state_dict(torch.load('/content/final_mode_vggresnet.pth', weights_only=True))
model = model.to(device)

## Load the test dataset file using upload file option and give the path below

In [16]:
test_path_degraded = 'paste/degraded/test/path/here'
test_path_gt = 'paste/gt/test/path/here'

In [17]:
class CustomDenoisedDataset(Dataset):
    def __init__(self, denoised_root, gt_root, transform=None, mode="train"):
        """
        Dataset class that loads denoised images and matches them with GT clean images for processing.

        Args:
            denoised_root (string): Directory containing denoised images organized by classes and subclasses.
            gt_root (string): Directory containing ground truth (GT) clean images organized by classes and subclasses.
            transform (callable, optional): Transformations to apply to images.
            mode (string): Indicates if the dataset is for 'train' or 'val' data.
        """
        self.data = []
        self.transform = transform
        self.mode = mode.capitalize()  # Ensure mode is in correct format (e.g., 'Train' or 'Val')

        # Traverse classes and subclasses in the denoised directory
        for class_name in os.listdir(denoised_root):
            class_denoised_dir = os.path.join(denoised_root, class_name, "Train", "Degraded_image")

            # Traverse each subclass within the class
            if not os.path.exists(class_denoised_dir):
                continue  # Skip if no directory exists for this class

            for subclass_name in os.listdir(class_denoised_dir):
                subclass_dir = os.path.join(class_denoised_dir, subclass_name)

                denoised_paths = glob(os.path.join(subclass_dir, "*.png"))  # All images in the subclass

                for denoised_path in denoised_paths:
                    # Match GT and Mask paths based on the current mode (train or val)
                    gt_path = denoised_path.replace(denoised_root, gt_root).replace("Degraded_image", "GT_clean_image")
                    mask_path = denoised_path.replace(denoised_root, gt_root).replace("Degraded_image", "Defect_mask").replace(".png", "_mask.png")

                    # Adjust paths for validation mode
                    if self.mode == "Val":
                        gt_path = gt_path.replace("Train", "Val")
                        mask_path = mask_path.replace("Train", "Val")

                    # Include the image only if the matching GT and mask images exist
                    if os.path.exists(gt_path) and os.path.exists(mask_path):
                        self.data.append((denoised_path, gt_path, mask_path, class_name, subclass_name))
        # Encode classes to integers for embedding
        self.class_to_idx = {cls: idx for idx, cls in enumerate(sorted(set([x[3] for x in self.data])))}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        denoised_path, gt_path, mask_path, class_name, subclass_name = self.data[idx]

        # Load images
        denoised_img = Image.open(denoised_path).convert("RGB")
        gt_img = Image.open(gt_path).convert("RGB")
        mask_img = Image.open(mask_path).convert("RGB")

        # Apply transformations, if any
        if self.transform:
            denoised_img = self.transform(denoised_img)
            gt_img = self.transform(gt_img)
            mask_img = self.transform(mask_img)

        # Get class index
        class_idx = self.class_to_idx[class_name]

        return denoised_img, gt_img, mask_img, class_idx, subclass_name

# Step 3: Define Transformations
transform = T.Compose([
    T.Resize((256, 256)),  # Resize to a manageable size for processing
    T.ToTensor(),
])

# Step 4: Initialize Dataset and DataLoader
denoised_root = "/content/denoised_test_images"
gt_root = "/content/Denoising_Dataset_train_val"
val_dataset = CustomDenoisedDataset(denoised_root=denoised_root, gt_root=gt_root, transform=transform, mode='val')
print("Dataset length:", len(val_dataset))
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)


Dataset length: 88


In [18]:
from collections import defaultdict

In [40]:
! rm -rf /content/output_dir

In [41]:
output_dir = "/content/output_dir"
os.makedirs(output_dir, exist_ok=True)

psnr_values = defaultdict(list)
ssim_values = defaultdict(list)

psnr_values_masked = defaultdict(list)
ssim_values_masked = defaultdict(list)

# Set model to evaluation mode
model.eval()
with torch.no_grad():
    for denoised_img, gt_img, mask_img, class_idx, subclass_name in val_loader:
        denoised_img, gt_img, mask_img, class_idx = denoised_img.to(device), gt_img.to(device), mask_img.to(device), class_idx.to(device)

        # Get model output for the denoised image
        model_output = model(denoised_img, class_idx)


        # Loop through batch to calculate metrics for each class
        for i in range(denoised_img.size(0)):
            # Convert images to numpy for PSNR and SSIM calculation
            denoised_img_np = denoised_img[i].cpu().numpy().transpose(1, 2, 0)
            gt_img_np = gt_img[i].cpu().numpy().transpose(1, 2, 0)
            model_output_np = model_output[i].cpu().numpy().transpose(1, 2, 0)
            defect_mask_np = mask_img[i].cpu().numpy().transpose(1, 2, 0)

            # Apply defect mask to model output

            model_output_defect = cv2.bitwise_and(model_output_np, defect_mask_np)
            gt_masked = cv2.bitwise_and(gt_img_np, defect_mask_np)

            # Calculate metrics
            psnr_value = psnr(gt_img_np, model_output_np, data_range=1)
            ssim_value = ssim(gt_img_np, model_output_np, multichannel=True, win_size=3, data_range=1)

            psnr_value_defect = psnr(gt_masked, model_output_defect, data_range=1)
            ssim_value_defect = ssim(gt_masked, model_output_defect, multichannel=True, win_size=3, data_range=1)

            # Append to respective class metric lists
            class_label = list(val_dataset.class_to_idx.keys())[class_idx[i].item()]
            psnr_values[class_label].append(psnr_value)
            ssim_values[class_label].append(ssim_value)

            psnr_values_masked[class_label].append(psnr_value_defect)
            ssim_values_masked[class_label].append(ssim_value_defect)

            subclass_label = subclass_name[i]
            output_class_dir = os.path.join(output_dir, class_label, subclass_label)
            os.makedirs(output_class_dir, exist_ok=True)

            # Save the model output and defect-masked output images
            output_img_path = os.path.join(output_class_dir, f"output_{i}.png")
            output_img_defect_path = os.path.join(output_class_dir, f"output_defect_{i}.png")

            # Convert model output images back to PIL format
            plt.imsave(output_img_path, np.clip(model_output_np, 0, 1))
            plt.imsave(output_img_defect_path, np.clip(model_output_defect, 0, 1))

In [42]:
# Calculate and print average PSNR and SSIM values for each class and overall
avg_psnr_values = {}
avg_ssim_values = {}
avg_psnr_values_masked = {}
avg_ssim_values_masked = {}

# Per-class averages
for class_label in psnr_values:
    avg_psnr_values[class_label] = sum(psnr_values[class_label]) / len(psnr_values[class_label])
    avg_ssim_values[class_label] = sum(ssim_values[class_label]) / len(ssim_values[class_label])

    avg_psnr_values_masked[class_label] = sum(psnr_values_masked[class_label]) / len(psnr_values_masked[class_label])
    avg_ssim_values_masked[class_label] = sum(ssim_values_masked[class_label]) / len(ssim_values_masked[class_label])

    print(f"Class: {class_label}")
    print(f"  Average PSNR (Full Image): {avg_psnr_values[class_label]:.4f}")
    print(f"  Average SSIM (Full Image): {avg_ssim_values[class_label]:.4f}")
    print(f"  Average PSNR (Defect Masked): {avg_psnr_values_masked[class_label]:.4f}")
    print(f"  Average SSIM (Defect Masked): {avg_ssim_values_masked[class_label]:.4f}")
    print("----------------------------------------------------")

# Overall averages
total_psnr = sum([psnr for psnr_list in psnr_values.values() for psnr in psnr_list])
total_ssim = sum([ssim for ssim_list in ssim_values.values() for ssim in ssim_list])
total_psnr_masked = sum([psnr_masked for psnr_list in psnr_values_masked.values() for psnr_masked in psnr_list])
total_ssim_masked = sum([ssim_masked for ssim_list in ssim_values_masked.values() for ssim_masked in ssim_list])

num_psnr = sum([len(psnr_list) for psnr_list in psnr_values.values()])
num_ssim = sum([len(ssim_list) for ssim_list in ssim_values.values()])
num_psnr_masked = sum([len(psnr_list) for psnr_list in psnr_values_masked.values()])
num_ssim_masked = sum([len(ssim_list) for ssim_list in ssim_values_masked.values()])

overall_avg_psnr = total_psnr / num_psnr
overall_avg_ssim = total_ssim / num_ssim
overall_avg_psnr_masked = total_psnr_masked / num_psnr_masked
overall_avg_ssim_masked = total_ssim_masked / num_ssim_masked

print("Overall Averages:")
print(f"  Average PSNR (Full Image): {overall_avg_psnr:.4f}")
print(f"  Average SSIM (Full Image): {overall_avg_ssim:.4f}")
print(f"  Average PSNR (Defect Masked): {overall_avg_psnr_masked:.4f}")
print(f"  Average SSIM (Defect Masked): {overall_avg_ssim_masked:.4f}")


Class: screw
  Average PSNR (Full Image): 21.3921
  Average SSIM (Full Image): 0.7939
  Average PSNR (Defect Masked): 42.2090
  Average SSIM (Defect Masked): 0.9981
----------------------------------------------------
Class: metal_nut
  Average PSNR (Full Image): 25.2789
  Average SSIM (Full Image): 0.7740
  Average PSNR (Defect Masked): 38.2297
  Average SSIM (Defect Masked): 0.9851
----------------------------------------------------
Class: pill
  Average PSNR (Full Image): 27.7351
  Average SSIM (Full Image): 0.8182
  Average PSNR (Defect Masked): 43.8457
  Average SSIM (Defect Masked): 0.9911
----------------------------------------------------
Class: transistor
  Average PSNR (Full Image): 26.0093
  Average SSIM (Full Image): 0.8005
  Average PSNR (Defect Masked): 37.1148
  Average SSIM (Defect Masked): 0.9350
----------------------------------------------------
Class: bottle
  Average PSNR (Full Image): 13.9049
  Average SSIM (Full Image): 0.4369
  Average PSNR (Defect Masked): 3